In [2]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import oge.download_data as download_data
import oge.load_data as load_data
from oge.column_checks import get_dtypes, apply_dtypes
from oge.filepaths import *
import oge.impute_hourly_profiles as impute_hourly_profiles
import oge.data_cleaning as data_cleaning
import oge.output_data as output_data
import oge.emissions as emissions
import oge.validation as validation
import oge.gross_to_net_generation as gross_to_net_generation
import oge.eia930 as eia930
from oge.logging_util import get_logger, configure_root_logger

configure_root_logger()
logger = get_logger("test")


year = 2018
path_prefix = f"{year}/"

In [9]:
cems = pd.read_csv(
    outputs_folder("2018/cems_cleaned_2018.csv.zip"),
    compression="zip",
    dtype=get_dtypes(),
)
eia923_allocated = pd.read_csv(
    outputs_folder("2018/eia923_allocated_2018.csv.zip"),
    compression="zip",
    dtype=get_dtypes(),
)
primary_fuel_table = pd.read_csv(
    outputs_folder("2018/primary_fuel_table_2018.csv.zip"),
    compression="zip",
    dtype=get_dtypes(),
)

In [13]:
cems["report_date"] = cems["report_date"].astype("datetime64[s]")

In [15]:
eia923_allocated["report_date"] = eia923_allocated["report_date"].astype(
    "datetime64[s]"
)

In [16]:
gtn_conversions = gross_to_net_generation.calculate_gross_to_net_conversion_factors(
    cems, eia923_allocated, primary_fuel_table, year
)
gtn_conversions

2024-06-29 12:55:04 [INFO] oge.oge.validation:480 Checking that all data has an associated `subplant_id`...  
2024-06-29 12:55:04 [INFO] oge.oge.validation:493 OK
2024-06-29 12:56:22 [INFO] oge.oge.validation:480 Checking that all data has an associated `subplant_id`...  
2024-06-29 12:56:22 [INFO] oge.oge.validation:493 OK
2024-06-29 12:56:28 [WARNING] oge.oge.validation:520 There are 1235 subplants at 517 plants for which there is zero gross generation associated with positive net generation.
2024-06-29 12:56:31 [WARNING] oge.oge.validation:523 
   plant_id_eia  subplant_id report_date  gross_generation_mwh  net_generation_mwh data_source ba_code
0             7            1  2018-01-01                   0.0             14990.0        both    SOCO
1             7            1  2018-02-01                   0.0             13488.0        both    SOCO
2             7            1  2018-03-01                   0.0              5462.0        both    SOCO
3             7            1  2018

,plant_id_eia,subplant_id,report_date,hours_in_month_subplant,gross_generation_mwh,minimum_gross_generation_mwh,maximum_gross_generation_mwh,net_generation_mwh,data_source,capacity_mw,monthly_subplant_ratio,monthly_subplant_shift_mw,annual_subplant_ratio,annual_subplant_shift_mw,monthly_plant_ratio,hours_in_month_plant,annual_plant_ratio,annual_plant_shift_mw,annual_fleet_ratio,subplant_regression_ratio,subplant_regression_shift_mw,subplant_regression_rsq_adj,plant_regression_ratio,plant_regression_shift_mw,plant_regression_rsq_adj,default_gtn_ratio
0,2,1,2018-01-01,NaN,NaN,NaN,NaN,-35.5,eia_only,53.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
1,2,1,2018-02-01,NaN,NaN,NaN,NaN,-35.2,eia_only,53.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
2,2,1,2018-03-01,NaN,NaN,NaN,NaN,-35.8,eia_only,53.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
3,2,1,2018-04-01,NaN,NaN,NaN,NaN,-37.8,eia_only,53.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
4,2,1,2018-05-01,NaN,NaN,NaN,NaN,-42.3,eia_only,53.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240369,62778,1,2018-12-01,NaN,NaN,NaN,NaN,445.4,eia_only,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
240370,62784,1,2018-12-01,NaN,NaN,NaN,NaN,13.0,eia_only,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
240371,62785,1,2018-12-01,NaN,NaN,NaN,NaN,9.0,eia_only,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99
240372,62805,1,2018-12-01,NaN,NaN,NaN,NaN,99.0,eia_only,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99


In [18]:
factors_to_use = gross_to_net_generation.filter_gtn_conversion_factors(gtn_conversions)

In [19]:
factors_to_use

,plant_id_eia,subplant_id,report_date,data_source,gross_generation_mwh,net_generation_mwh,minimum_gross_generation_mwh,maximum_gross_generation_mwh,capacity_mw,annual_subplant_ratio,annual_plant_ratio,annual_fleet_ratio,annual_subplant_shift_mw,annual_plant_shift_mw,default_gtn_ratio
0,2,1,2018-01-01,eia_only,NaN,-35.5,NaN,NaN,53.9,NaN,NaN,NaN,NaN,NaN,0.99
1,2,1,2018-02-01,eia_only,NaN,-35.2,NaN,NaN,53.9,NaN,NaN,NaN,NaN,NaN,0.99
2,2,1,2018-03-01,eia_only,NaN,-35.8,NaN,NaN,53.9,NaN,NaN,NaN,NaN,NaN,0.99
3,2,1,2018-04-01,eia_only,NaN,-37.8,NaN,NaN,53.9,NaN,NaN,NaN,NaN,NaN,0.99
4,2,1,2018-05-01,eia_only,NaN,-42.3,NaN,NaN,53.9,NaN,NaN,NaN,NaN,NaN,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240369,62778,1,2018-12-01,eia_only,NaN,445.4,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.99
240370,62784,1,2018-12-01,eia_only,NaN,13.0,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,0.99
240371,62785,1,2018-12-01,eia_only,NaN,9.0,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,0.99
240372,62805,1,2018-12-01,eia_only,NaN,99.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,0.99
